# Deploy LLama-2 7B on Vertex AI and interact it through a Gradio UI

<table align="left">
  <td>
    <a href="https://github.com/huggingface/Google-Cloud-Containers/blob/main/examples/vertex-ai/notebooks/deploy-llama-2-on-vertex-ai.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
</table>
<br/><br/><br/>

[LLama-2-7b](https://huggingface.co/meta-llama/Llama-2-7b-chat-hf) is a a Llama 2 model with 7B parameters fine-tuned for chat instructions. You can find more information about it on the blogpost [Llama 2 is here - get it on Hugging Face](https://huggingface.co/blog/llama2).

In this tutorial you will learn how to deploy [meta-llama/Llama-2-7b-chat-hf](https://huggingface.co/meta-llama/Llama-2-7b-chat-hf) on Vertex AI Endpoints. We are going to use the Hugging Face [Text Generation Inference (TGI)](https://github.com/huggingface/text-generation-inference) container which is a scalelable, optimized solution for deploying and serving Large Language Models (LLMs). You can now find all Hugging Face containers on [Google Cloud](https://cloud.google.com/deep-learning-containers/docs/choosing-container#hugging-face).

What you'll learn in this blog:

1. [Setup development environment](#1-setup-development-environment)
2. [Configure gcloud CLI](#2-configure-gcloud-cli)
3. [Initialize Vertex AI SDK](#3-initialize-vertex-ai-sdk)
4. [Deploy Llama-2-7B on Vertex AI](#4-deploy-llama-2-7b-on-vertex-ai)
5. [Run Inference with deployed Model](#5-run-inference-with-deployed-model)
6. [Create Chatbot UI using Gradio](#6-create-chatbot-ui-using-gradio)
7. [Cleaning Up Resources](#7-cleaning-up-resources)

## 1. Setup development environment

We are going to use the `Vertex AI` python SDK to deploy [meta-llama/Llama-2-7b-chat-hf](https://huggingface.co/meta-llama/Llama-2-7b-chat-hf) on Vertex AI. You need to have a GCP project and an account with the necessary permissions to create resources in the project. 

Before we can install the packages we need to install the `gcloud CLI`. Instructions can be found here: https://cloud.google.com/sdk/docs/install

In [ ]:
# Install the required packages for the notebook
! pip install --upgrade --quiet google-cloud-aiplatform google-cloud-storage "google-auth>=2.23.3"

## 2. Configure gcloud CLI

We need to authenticate with Google Cloud SDK to use the Vertex AI services. Run the following command to authenticate with Google Cloud SDK.

```bash
gcloud auth login 
gcloud auth application-default login
```

**If you don't know your project ID**, try the following:
* Run `gcloud config list`.
* Run `gcloud projects list`.
* See the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113)


In [ ]:
PROJECT_ID = "gcp-partnership-412108"  # @param {type:"string"}
REGION = "us-central1"  # @param {type: "string"}
BUCKET_URI = f"gs://vertexai-{PROJECT_ID}-tgi"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID} --quiet
# Set the region
! gcloud config set ai/region {REGION} --quiet
# create the bucket if it doesn't exist
! gsutil mb -l {REGION} -p {PROJECT_ID} {BUCKET_URI}

## 3. Initialize Vertex AI SDK 

The following set of constants will be used to create names and display names of Vertex AI Prediction resources like models, endpoints, and model deployments.

In [ ]:
# set model names and version
MODEL_NAME = "Llama-2-7b" # @param {type:"string"}
MODEL_VERSION = "v01" # @param {type: "string"}
MODEL_DISPLAY_NAME = f"TGI-{MODEL_NAME}-{MODEL_VERSION}" # @param {type:"string"}
ENDPOINT_DISPLAY_NAME = f"endpoint-{MODEL_NAME}-{MODEL_VERSION}" # @param {type:"string"}

# Set the TGI serving container image uri, selected from https://cloud.google.com/deep-learning-containers/docs/choosing-container#hugging-face
SERVING_CONTAINER_IMAGE_URI = "us-docker.pkg.dev/deeplearning-platform-release/gcr.io/huggingface-text-generation-inference-cu121.1-4.ubuntu2204.py310"

In [ ]:
from google.cloud import aiplatform

# Initialize the Vertex AI SDK
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

## 4. Deploy Llama-2-7B on Vertex AI

To deploy [Llama-2-7B](https://huggingface.co/meta-llama/Llama-2-7b-chat-hf) on Vertex AI, we first need to first upload the model to [Vertex AI Model Registry](https://cloud.google.com/vertex-ai/docs/model-registry/introduction).

In [ ]:
model = aiplatform.Model.upload(
    display_name=MODEL_DISPLAY_NAME,
    serving_container_image_uri=SERVING_CONTAINER_IMAGE_URI,
    serving_container_environment_variables={
        "MODEL_ID": "meta-llama/Llama-2-7b-chat-hf", # Hugging Face model ID
        "NUM_SHARD": "1", 
        "MAX_INPUT_LENGTH": "512", 
        "MAX_TOTAL_TOKENS": "1024", 
        "HUGGING_FACE_HUB_TOKEN": "", # Replace with your Hugging Face Hub token
        },
    serving_container_ports=[80],
)


model.wait()

print(model.display_name)
print(model.resource_name)

Once the model is uploaded, now we can deploy it on Vertex AI Endpoints. 
First, we need to create an endpoint and then deploy the model to the endpoint. Here you also need to choose the hardware configuration for the deployment. 

The deployment will take ~20-25 minutes. You can check the status of the deployment in the Google cloud console.

In [ ]:
machine_type = 'g2-standard-4' # L4 GPUs
endpoint = aiplatform.Endpoint.create(display_name=ENDPOINT_DISPLAY_NAME)

deployed_model = model.deploy(
    endpoint=endpoint,
    deployed_model_display_name=MODEL_NAME, # The display name of the deployed model
    machine_type=machine_type,              # type of machine, read more here: https://cloud.google.com/vertex-ai/docs/predictions/configure-compute
    accelerator_type="NVIDIA_L4",           # Hardware accelerator 
    accelerator_count=1,                    # Number of accelerators to attach to a worker replica.
    min_replica_count=1,                    # The minimum number of machine replicas this deployed model will be always deployed on.
    sync=True,                              # Whether to execute this method synchronously.
)

## 5. Run Inference with deployed Model
Awesome! We have successfully deployed the Llama-2-7B model on Vertex AI. Now let's run inference on our endpoint. We will use the `predict` method of the `Endpoint` class to run inference on the deployed model. We can inference with different parameters to impact the generation. Parameters can be defined as in the `parameters` attribute of the payload. You can find supported parameters in the [TGI documentation](https://huggingface.github.io/text-generation-inference/) defined under the `GenerateParameters` section.


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf", 
                                          token="Your Hugging Face Hub token here",)

messages = [
  {"role": "user", "content": "What is Hugging Face?"},
]
inputs = tokenizer.apply_chat_template(messages, tokenize=False)
res = deployed_model.predict(instances=[
  {"inputs": inputs, 
   "parameters": {"max_new_tokens": 256, "do_sample": True, "top_p": 0.7, "temparature": 1.0 }} # Generation arguments
  ]
)
print(res.predictions[0])


## 6. Create Chatbot UI using Gradio

We can create a simple chatbot UI using [Gradio](https://gradio.app/) to interact with our deployed model on Vertex AI Endpoints. Gradio is an open-source Python library that allows you to create UIs for your machine learning models with a few lines of code. You can find more information about Gradio in the [documentation](https://gradio.app/docs).

In [ ]:
# Install gradio for interactive UI
! pip install gradio

In [ ]:
import gradio as gr

# hyperparameters for llm
parameters = {
    "max_new_tokens": 256, 
    "do_sample": True, 
    "top_p": 0.7, 
    "temparature": 1.0 
  }

MAX_INPUT_TOKEN_LENGTH = 512 # Set also in the serving container

with gr.Blocks() as demo:
    gr.Markdown("## Chat with Llama-2-7b!")
    with gr.Column():
        chatbot = gr.Chatbot()
        with gr.Row():
            with gr.Column():
                message = gr.Textbox(label="Chat Message Box", placeholder="Chat Message Box", show_label=False)
            with gr.Column():
                with gr.Row():
                    submit = gr.Button("Submit")
                    clear = gr.Button("Clear")

    def respond(message, chat_history):
        conversation = []
        for user, assistant in chat_history:
            conversation.extend([{"role": "user", "content": user}, {"role": "assistant", "content": assistant}])
        conversation.append({"role": "user", "content": message})
        inputs = tokenizer.apply_chat_template(conversation, tokenize=False)
        # send request to endpoint
        llm_response = deployed_model.predict(instances=[{"inputs": inputs, 
                                               "parameters": parameters}])
        parsed_response = llm_response.predictions[0]
        chat_history.append((message, parsed_response))
        return "", chat_history

    submit.click(respond, [message, chatbot], [message, chatbot], queue=False)
    clear.click(lambda: None, None, chatbot, queue=False)

demo.launch(share=True)

## 7. Cleaning Up Resources

To avoid incurring charges to your Google Cloud Platform account for the resources used in this tutorial, you can delete the resources you created in this tutorial.

In [ ]:
deployed_model.undeploy_all()
deployed_model.delete()
model.delete()